In [62]:
from casatasks import tclean, split, listobs
from casatasks import uvcontsub, flagdata, gencal, plotweather, setjy, gaincal, bandpass, applycal, fluxscale
try:
    from casaplotms import plotms
except:
    import os
    os.system('pip install casaplotms')
    from casaplotms import plotms
from pathlib import Path
from casatools import image as IA
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import numpy as np
import io
import base64
from os import path, makedirs, remove, getcwd
import glob
from shutil import rmtree

In [2]:
# helpers

def __clear_tmp(wd,cimagename):
    filelist = glob.glob(path.join(wd, cimagename+'.*'))
    for f in filelist:
        try:
            rmtree(f)
        except OSError:
            remove(f)
def save_fig(plt, fig, kind='base64', output='output.jpg'):
    
    if kind == 'base64':
        buf = io.BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight',
                    transparent=True, pad_inches=0)
        buf.seek(0)
        string = base64.b64encode(buf.read())
        plt.close()
        return string
    elif kind == 'plot':
        plt.show()
        return 'plotted'
    else :
        if not path.exists('output'):
            makedirs('output')
        newPath = 'output/'+output
        opt = newPath
        if path.exists(newPath):
            numb = 1
            while path.exists(newPath):
                newPath = "{0}_{2}{1}".format(
                    *path.splitext(opt) + (numb,))
                try :
                    if path.exists(newPath):
                        numb += 1 
                except:
                    pass               
        fig.savefig(newPath, format=kind, bbox_inches='tight',
                    pad_inches=0)
        print("saved {}".format(newPath))
        plt.close()
        return newPath


In [4]:
cwd = getcwd()
csource = str(Path.home()) + '/CASA/tests/test_d/day2_TDEM0003_10s_norx' # ms file as visibility input
csource_stem = str(Path(csource).stem) # filename for p1 creations

work_folder = str(Path.home()) + '/CASA/tests/test_d/tmp/'+csource_stem+'/' # all creations inside this
if not path.exists(work_folder):
            makedirs(work_folder)
csource_smoothed = work_folder+csource_stem+'_p1.ms' # created by p1
cimagename=str(Path(csource_smoothed).stem) # created by p1 during tclean
#rest_freq = '372.67249GHz'
science_imagename='IRCp_science'
science_vis = csource + '.contsub'
niter=5000

In [ ]:
# time v elevation
# time v amplitude
plotms(vis=csource,
xaxis='time',yaxis='elevation',correlation='RR,LL',
       #xaxis='time',yaxis='amp',correlation='RR,LL',
       avgchannel='64',spw='0:4~60', coloraxis='field')

In [ ]:
# time v amplitude
# uv_distance v amplitude
plotms(vis=csource,field='3',
       xaxis='time',yaxis='amp',correlation='RR,LL',
       # xaxis='uvdist',yaxis='amp',correlation='RR,LL',
       avgchannel='64',spw='0~1:4~60', coloraxis='spw')

In [15]:
# flag the visibility data for calibration
flagdata(vis=csource,
     mode='list',
     inpfile=["field='2,3' antenna='ea12' timerange='03:41:00~04:10:00'", # flag for amplitude vs time cal
              "field='2,3' antenna='ea07,ea08' timerange='03:21:40~04:10:00' spw='1'"]) # flag for elev vs t cal

{}

In [21]:
# baseline correction 
# offset calculation (VLA) baselines

gencal(vis=csource,caltable='antpos.cal',
       caltype='antpos',
       antenna='')


In [26]:
# TODO: Note - only for VLA telescope
# Gaincurves and antenna efficiency


#gencal(vis=csource,caltable='gaincurve.cal',
#       caltype='gceff')
myTau = plotweather(vis=csource, doPlot=True)

In [27]:
gencal(vis=csource,caltable='opacity.cal',
       caltype='opac',
       spw='0,1',
       parameter=myTau)

In [ ]:
# flux model for calibration
setjy(vis=csource,listmodels=True)
setjy(vis=csource,field='7',spw='0~1',scalebychan=True,model='3C286_A.im') # field 7 flux calib

In [44]:
# investigate phase variation as a function of channel and time

plotms(vis=csource,field='5',
       xaxis='time',#xaxis='channel',
       yaxis='phase',
       correlation='RR',#correlation='LL',
       avgchannel='64', # for time vs phase
       # avgtime='1e8' #for ch vs phase
       spw='0:4~60',# spw='1:4~60', 
       coloraxis='antenna2',
       antenna='ea02'#, antenna='ea02&ea23', 
      )

2021-12-23 19:56:03	WARN	MSTransformManager::initDataSelectionParams	Number of selected channels 57 for SPW 0 is smaller than specified chanbin 64
2021-12-23 19:56:03	WARN	MSTransformManager::initDataSelectionParams+	Setting chanbin to 57 for SPW 0


In [48]:
# Delay Calibration due to phase variation

gaincal(vis=csource, caltable='delays.cal', field='5', 
        refant='ea02', gaintype='K', gaintable=['antpos.cal','opacity.cal'])

In [50]:
# Bandpass Calibration
gaincal(vis=csource,
        caltable='bpphase.gcal',
        field='5',spw='0~1:20~40',
        refant='ea02',calmode='p',solint='int',minsnr=2.0,
        gaintable=['antpos.cal','opacity.cal','delays.cal'])
plotms(vis='bpphase.gcal',gridrows=3,gridcols=3,xaxis='time',yaxis='phase',
       iteraxis='antenna',coloraxis='corr',plotrange=[0,0,-180,180])

In [53]:
bandpass(vis=csource,caltable='bandpass.bcal',field='5',
        refant='ea02',solint='inf',solnorm=False,
        gaintable=['antpos.cal','opacity.cal','delays.cal','bpphase.gcal'])

In [56]:
applycal(vis=csource,field='5',
        gaintable=['antpos.cal','opacity.cal','delays.cal','bandpass.bcal'],
        gainfield=['','','5','5'],calwt=False)

In [58]:
# Gain Calibration

gaincal(vis=csource,caltable='intphase.gcal',field='2,5,7',
        spw='0~1:4~60',
        solint='int',calmode='p', # will derive a single phase solution for each integration (as data was avgd for 10s)
        refant='ea02',minsnr=2.0,
        gaintable=['antpos.cal','opacity.cal','delays.cal','bandpass.bcal'])

2 of 36 solutions flagged due to SNR < 2 in spw=0 at 2010/04/26/03:22:16.0
1 of 36 solutions flagged due to SNR < 2 in spw=0 at 2010/04/26/03:22:26.0
1 of 36 solutions flagged due to SNR < 2 in spw=0 at 2010/04/26/03:22:36.0
1 of 36 solutions flagged due to SNR < 2 in spw=0 at 2010/04/26/03:22:56.0
1 of 30 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/03:21:57.1
1 of 32 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/03:22:16.0
2 of 32 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/03:22:26.0
2 of 32 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/03:22:36.0
1 of 32 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/03:22:46.0
1 of 32 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/03:22:56.0
1 of 32 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/03:23:06.0
2 of 32 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/03:23:16.0
1 of 36 solutions flagged due to SNR < 2 in spw=0 at 2010/04/26/03:28:43.4
1 of 38 solutions flagged

1 of 38 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/05:24:22.0
1 of 38 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/05:30:58.0
1 of 38 solutions flagged due to SNR < 2 in spw=0 at 2010/04/26/05:43:49.0
1 of 38 solutions flagged due to SNR < 2 in spw=1 at 2010/04/26/05:43:56.0


In [60]:
gaincal(vis=csource,caltable='scanphase.gcal',field='2,5,7',
        spw='0~1:4~60',solint='inf',refant='ea02',minsnr=2.0,calmode='p',
        gaintable=['antpos.cal','opacity.cal','delays.cal','bandpass.bcal'])

In [69]:
gaincal(vis=csource,caltable='amp.gcal',field='2,5,7',
        spw='0~1:4~60',solint='inf',refant='ea02',minsnr=2.0,calmode='ap',
        gaintable=['antpos.cal','opacity.cal','delays.cal','bandpass.bcal','intphase.gcal'])

1 of 38 solutions flagged due to SNR < 2.000000000000 in spw=0 at 2010/04/26/03:29:14.7
1 of 34 solutions flagged due to SNR < 2.000000000000 in spw=1 at 2010/04/26/03:29:15.3
1 of 34 solutions flagged due to SNR < 2.000000000000 in spw=1 at 2010/04/26/03:35:44.2


In [70]:

fluxscale(vis=csource,
          caltable='amp.gcal',
          fluxtable='flux.cal',
          reference='7',incremental=True)
# derived flux densities

{'2': {'0': {'fluxd': array([0.27157678, 0.        , 0.        , 0.        ]),
   'fluxdErr': array([0.03269022, 0.        , 0.        , 0.        ]),
   'numSol': array([38.,  0.,  0.,  0.])},
  '1': {'fluxd': array([0.26827228, 0.        , 0.        , 0.        ]),
   'fluxdErr': array([0.03170749, 0.        , 0.        , 0.        ]),
   'numSol': array([38.,  0.,  0.,  0.])},
  'covarMat': array([[7.11465664e-03, 2.63269854e-01],
         [2.63269854e-01, 2.91588965e+04]]),
  'fieldName': 'J0954+1743',
  'fitFluxd': 0.2699194690852456,
  'fitFluxdErr': 6.24938056072569e-09,
  'fitRefFreq': 36349862201.616486,
  'spidx': array([-0.56876579,  5.38185402]),
  'spidxerr': array([1.00551157e-08, 2.03561500e-05])},
 '5': {'0': {'fluxd': array([31.08555924,  0.        ,  0.        ,  0.        ]),
   'fluxdErr': array([0.24362334, 0.        , 0.        , 0.        ]),
   'numSol': array([38.,  0.,  0.,  0.])},
  '1': {'fluxd': array([30.66536005,  0.        ,  0.        ,  0.        ]),
 